In [6]:
import dash
from dash import dcc, html, Input, Output
import pandas as pd
import plotly.express as px
import folium
from folium import IFrame, Popup
import dash_table
from sklearn.cluster import KMeans  # only import once
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier



# Initialisation de l'application Dash
app = dash.Dash(__name__)

# Charger les données
medianes = pd.read_csv('land-registry-house-prices-borough.csv')
age_batiment = pd.read_csv("london_houses.csv")
recycl = pd.read_csv("Household-rcycling-borough.csv")

# Nettoyage des données
medianes["Year"] = medianes["Year"].astype(str).str.extract(r'(\d{4})').astype(int)
medianes["Value"] = medianes["Value"].str.replace(",", "").astype(float)
age_batiment = age_batiment.rename(columns={'Price (£)': 'Price'})

# Moyenne des taux de recyclage par quartier
moyenne_recyclage = recycl.groupby("Area")["Recycling_Rates"].mean().reset_index()
moyenne_recyclage = moyenne_recyclage.sort_values(by="Recycling_Rates", ascending=False)

# Coordonnées des quartiers de Londres
london_boroughs_coords = {
    "City of London": [51.51279, -0.09184],
    "Barking and Dagenham": [51.5362, 0.1275],
    "Barnet": [51.6255, -0.1514],
    # Ajoutez ici les autres quartiers...
}

def generate_recycling_map():
    # Calcul du taux moyen de recyclage par quartier
    avg_recycling = recycl.groupby("Area")["Recycling_Rates"].mean().to_dict()

    # Création de la carte centrée sur Londres
    m = folium.Map(location=[51.5074, -0.1278], zoom_start=10)

    # Fonction pour déterminer la couleur en fonction du taux de recyclage
    def get_recycling_color(rate):
        if rate <= 15:
            return "red"  # Faible
        elif rate <= 30:
            return "orange"  # Moyenne
        elif rate <= 45:
            return "yellow"  # Élevée
        else:
            return "green"  # Très élevée

    # Ajout de marqueurs pour chaque quartier
    for borough, coords in london_boroughs_coords.items():
        if borough in avg_recycling:
            mean_rate = avg_recycling[borough]
            
            # Création du graphique d’évolution avec Plotly
            borough_data = recycl[recycl["Area"] == borough]
            fig = px.line(
                borough_data,
                x="Year",
                y="Recycling_Rates",
                title=f"Évolution du taux de recyclage à {borough}",
                labels={"Year": "Année", "Recycling_Rates": "Taux de recyclage (%)"},
            )

            # Conversion du graphique en HTML
            iframe = IFrame(fig.to_html(full_html=False, include_plotlyjs='cdn'), width=500, height=300)
            popup = Popup(iframe, max_width=500)

            # Ajout d’un cercle sur la carte avec la couleur selon le taux de recyclage
            folium.CircleMarker(
                location=coords,
                radius=10,
                color="blue",
                fill=True,
                fill_color=get_recycling_color(mean_rate),  # Couleur selon le taux
                fill_opacity=0.6,
                popup=popup,
            ).add_to(m)

    # Ajout d’une légende manuelle
    legend_html = '''
    <div style="position: fixed; 
                bottom: 50px; left: 50px; width: 150px; height: 120px; 
                background-color: white; z-index:9999; font-size:14px;
                padding: 10px; border-radius: 5px;
                box-shadow: 2px 2px 5px rgba(0,0,0,0.3);">
        <b>Légende :</b><br>
        <i class="fa fa-circle" style="color:red"></i> Faible (≤15%)<br>
        <i class="fa fa-circle" style="color:orange"></i> Moyenne (≤30%)<br>
        <i class="fa fa-circle" style="color:yellow"></i> Élevée (≤45%)<br>
        <i class="fa fa-circle" style="color:green"></i> Très élevée (>45%)<br>
    </div>
    '''
    m.get_root().html.add_child(folium.Element(legend_html))

    # Sauvegarde de la carte en HTML
    map_path = "carte_recyclage.html"
    m.save(map_path)
    
    return map_path

#pour la section 3

def generate_cluster_repartition_and_intervals(kmeans_model, cluster_column_name):
    cluster_centroids = kmeans_model.cluster_centers_
    cluster_intervals = sorted(cluster_centroids.flatten())  # Tri des centroids pour obtenir des intervalles
    
    # Ajout des bornes minimales et maximales pour créer les intervalles de prix
    min_price, max_price = Prix2['Price'].min(), Prix2['Price'].max()
    cluster_intervals = [min_price] + cluster_intervals + [max_price]
    
    # Répartition des données dans chaque cluster
    cluster_repartition = Prix2[cluster_column_name].value_counts().sort_index()

    # Créer une liste des intervalles de prix
    price_intervals = [f"{cluster_intervals[i]} - {cluster_intervals[i + 1]}" for i in range(len(cluster_intervals) - 1)]

    # DataFrame pour la répartition des clusters et des intervalles de prix
    cluster_repartition_df = pd.DataFrame({
        "Cluster": cluster_repartition.index,
        "Count": cluster_repartition.values,
        "Price Interval": price_intervals[:len(cluster_repartition)]  # Ajuster la taille des intervalles
    })
    
    return cluster_repartition_df

X = Prix2[['Price']] 

# Create the KMeans model with 5 clusters
kmeans_5 = KMeans(n_clusters=5, random_state=42)
Prix2['Price Category (5 Clusters)'] = kmeans_5.fit_predict(X)
# For kmeans_4 with 4 clusters
kmeans_4 = KMeans(n_clusters=4, random_state=42)
Prix2['Price Category (4 Clusters)'] = kmeans_4.fit_predict(X)

# For kmeans_6 with 6 clusters
kmeans_6 = KMeans(n_clusters=6, random_state=42)
Prix2['Price Category (6 Clusters)'] = kmeans_6.fit_predict(X)

# For kmeans_7 with 7 clusters
kmeans_7 = KMeans(n_clusters=7, random_state=42)
Prix2['Price Category (7 Clusters)'] = kmeans_7.fit_predict(X)

# For kmeans_8 with 8 clusters
kmeans_8 = KMeans(n_clusters=8, random_state=42)
Prix2['Price Category (8 Clusters)'] = kmeans_8.fit_predict(X)



# 1. Générer les tableaux pour chaque nombre de clusters
cluster_4_df = generate_cluster_repartition_and_intervals(kmeans_4, 'Price Category (4 Clusters)')
cluster_5_df = generate_cluster_repartition_and_intervals(kmeans_5, 'Price Category (5 Clusters)')
cluster_6_df = generate_cluster_repartition_and_intervals(kmeans_6, 'Price Category (6 Clusters)')
cluster_7_df = generate_cluster_repartition_and_intervals(kmeans_7, 'Price Category (7 Clusters)')
cluster_8_df = generate_cluster_repartition_and_intervals(kmeans_8, 'Price Category (8 Clusters)')


cluster_data = {
    "4 Clusters": cluster_4_df,
    "5 Clusters": cluster_5_df,
    "6 Clusters": cluster_6_df,
    "7 Clusters": cluster_7_df,
    "8 Clusters": cluster_8_df
}

# Répartition des données d'apprentissage pour chaque catégorie de prix (clusters)
y_4_clusters = Prix2['Price Category (4 Clusters)']
y_5_clusters = Prix2['Price Category (5 Clusters)']
y_6_clusters = Prix2['Price Category (6 Clusters)']
y_7_clusters = Prix2['Price Category (7 Clusters)']
y_8_clusters = Prix2['Price Category (8 Clusters)']


# Sélectionner les colonnes que vous souhaitez inclure dans X
features = ['House_Type', 'Area_in_sq_ft', 'No__of_Bedrooms', 
            'No__of_Bathrooms', 'No__of_Receptions', 'Location']

X = Prix2[features]

# Encoder les variables catégorielles (exemple pour 'House_Type' et 'Location')
label_encoder = LabelEncoder()

# Applique l'encodage sur les variables catégorielles
X['House_Type'] = label_encoder.fit_transform(X['House_Type'])
X['Location'] = label_encoder.fit_transform(X['Location'])

# Pour 'Property_Name', vous pouvez faire de même si vous voulez l'utiliser aussi.
# Notez que l'encodage de 'Property_Name' peut ne pas être utile selon la nature de cette variable,
# mais si vous voulez l'utiliser, vous pouvez aussi appliquer un encodage similaire.

# Séparation des données en train et test pour les différentes catégories de prix (clusters)
X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(X, y_4_clusters, test_size=0.2, random_state=42)
X_train_5, X_test_5, y_train_5, y_test_5 = train_test_split(X, y_5_clusters, test_size=0.2, random_state=42)
X_train_6, X_test_6, y_train_6, y_test_6 = train_test_split(X, y_6_clusters, test_size=0.2, random_state=42)
X_train_7, X_test_7, y_train_7, y_test_7 = train_test_split(X, y_7_clusters, test_size=0.2, random_state=42)
X_train_8, X_test_8, y_train_8, y_test_8 = train_test_split(X, y_8_clusters, test_size=0.2, random_state=42)

# Modèles de classification
models = {
    "XGBoost": XGBClassifier(n_estimators=1000, max_depth=10, learning_rate=0.1, random_state=42),
    "LightGBM": lgb.LGBMClassifier(n_estimators=1000, max_depth=10, learning_rate=0.1, random_state=42, verbose=-1),
    "Random Forest": RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42),
    "Decision Tree": DecisionTreeClassifier(max_depth=10, random_state=42)
}

# Stockage des performances
results = []

for name, model in models.items():
    for decoupage, X_train, X_test, y_train, y_test in [
        ("4 Clusters", X_train_4, X_test_4, y_train_4, y_test_4),
        ("5 Clusters", X_train_5, X_test_5, y_train_5, y_test_5),
        ("6 Clusters", X_train_6, X_test_6, y_train_6, y_test_6),
        ("7 Clusters", X_train_7, X_test_7, y_train_7, y_test_7),
        ("8 Clusters", X_train_8, X_test_8, y_train_8, y_test_8)
    ]:
        # Entraînement
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Évaluation
        acc = accuracy_score(y_test, y_pred)
        results.append((name, decoupage, acc))

# Création d'un DataFrame pour stocker les résultats
df_results = pd.DataFrame(results, columns=['Modèle', 'Découpage', 'Accuracy'])



# Entraîner le modèle RandomForest sur les données des 5 clusters
rf_model_5 = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model_5.fit(X_train_5, y_train_5)

# Extraire l'importance des variables
feature_importances = rf_model_5.feature_importances_

# Créer un DataFrame pour afficher les résultats
importance_df = pd.DataFrame({
    'Feature': X_train_5.columns,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

# Créer un graphique de l'importance des variables
importance_fig = px.bar(importance_df, x='Feature', y='Importance', 
                        title="Importance des variables pour la classification avec Random Forest (5 Clusters)",
                        labels={'Importance': 'Importance', 'Feature': 'Variable'},
                        color='Importance', color_continuous_scale='Viridis')



# Layout de l'application Dash
app.layout = html.Div([
    html.H1("Analyse des logements à Londres"),
    html.P("Ce projet vise à analyser le marché du logement à Londres en exploitant diverses sources de données. L'objectif est de comprendre l'évolution des prix, des ventes et des caractéristiques des logements dans les différents quartiers de la ville et d'identifier des comportements similaires entre quartiers."),
    dcc.Markdown('''
    L'analyse s'est faite en plusieurs étapes : 

    - **Statistiques descriptives** : étude de l'évolution des prix par quartier, du nombre de ventes et des potentiels facteurs ayant une influence sur le prix des logements.
    - **Clustering** : L'objectif ici est de regrouper des quartiers ensemble autour de certaines caractéristiques.
    - **Classification** : identification des caractéristiques permettant de prédire l’appartenance d’un quartier à une tranche de prix donnée.

    Les données utilisées proviennent de sources officielles et ouvertes, notamment :
    - **Housing-sales-borough.csv** : ventes immobilières par quartier (1995-2014).
    - **Land-registry-house-prices-borough.csv** : prix médians des logements (1995-2017).
    - **Dclg-affordable-housing-borough.csv** : logements abordables construits par année et quartier.
    - **Household-recycling-borough.csv** : taux de recyclage par quartier (2003-2023).
    - **Tenure-households-borough.csv & Tenure-population-borough.csv** : répartition des logements et population par type d’occupation (2008-2018).
    - **London.csv & London_houses.csv** : caractéristiques détaillées des logements londoniens (prix, superficie, nombre de chambres, etc.).

    Cette analyse a pour objectif global de comprendre les dynamiques du marché des logements londoniens.
    '''),
    html.H2("1. Statistiques descriptives"),
    html.H3("1.1 Évolution des prix des logements"),
    dcc.Graph(id='prix-maisons-graph'),  # Graphique 1

    html.P("Sélectionnez un quartier pour voir l'évolution des prix."),
    dcc.Dropdown(
        id='dropdown-quartier',
        options=[{'label': area, 'value': area} for area in medianes["Area"].unique()],
        value='London',  # Quartier par défaut
        style={'width': '50%'}
    ),
    dcc.Graph(id='graph-quartier'),  # Graphique 2

    html.P("Ce graphique montre la distribution des prix des logements par quartier."),
    dcc.Graph(id='boxplot-prix'),  # Graphique 3

    html.H3("1.3 Taux de recyclage des logements de Londres"),
    html.P("Ce graphique montre le taux moyen de recyclage dans chaque quartier de Londres. "),
    dcc.Graph(id='graph-recyclage'),  # Graphique 5
    html.P("On remarque que le taux de recyclage varie d'un quartier à l'autre. Il serait intéressant de le visualiser sur une carte afin de détecter d'éventuelles tendances géographiques."),

    html.Iframe(id="recycling-map", srcDoc=open(generate_recycling_map(), "r", encoding="utf-8").read(), width="100%", height="600px", style={"border": "none"}),

    html.H2("2. Clustering"),
    html.P("L'objectif ici est de trouver des clusters de quartiers"),
    html.P("Ce jeu de données comprend les ventes, les prix médians et moyens des logements 3 fois par an de 1995 à 2022."),
    html.P("À ce jeu de données on a ajouté d'autres jeux de données sur les taux de recyclage déjà utilisé précédemment ainsi qu'un autre jeu de données sur les logements disponibles (lien)."),
    html.P("Des modifications ont été faites sur le nom des colonnes et les datasets ont été fusionnés. Ensuite les données ont été normalisées. Le clustering a été appliqué à l'aide de la méthode des k-means avec le k optimal trouvé grâce à la méthode du coude."),
    html.P("La méthode des k-means consiste à choisir k centroïdes initiaux, puis à assigner à chaque point restant le centroïde le plus proche. Puis le centroïde est recalculé. Une fois que l'algorithme converge on s'arrête."),
    html.P("Pour plus d'informations sur la méthode des k-means : "),
    html.A("Méthode des k-means  ", href= "https://mrmint.fr/algorithme-k-means"),
    
    html.P("Ici 3 clusters ont été établis. Regardons sur la carte si ils sont répartis d'une certaine manière."),
    
    html.H2("3. Classification des appartement par tranche de prix"),
    html.P("L'objectif ici est de de comparer différentes méthodes de classification sur le prix des appartement londonnien en ? "),
    html.P("Ce jeu de donnée comprend..."),
    
    html.H3("3.1 Clustering à l'aide de la méthode des k-means"),
    html.P("Le prix étant une variable continue nous avons dans un premier temps effectué un clustering à l'aide de la méthode des k-means sur les prix de vente des appartements afin de garder entre 4 et 8 catégories de prix"),
    html.P("Ces tableaux montrent la répartition des données pour les différents clusters des différents clustering"),
    html.H3("Sélectionnez un clustering"),
    
    dcc.Dropdown(
        id="cluster-dropdown",
        options=[{"label": key, "value": key} for key in cluster_data.keys()],
        value="4 Clusters",  # Valeur par défaut
        clearable=False
    ),

    dash_table.DataTable(
        id="cluster-table",
        style_table={'height': '300px', 'overflowY': 'auto'}),
    html.H3("3.2 Comparaison de différentes méthodes de classification"),
    html.P("Afin de comparer la précision des méthodes, l'ensemble des données a été séparé en ensemble d'apprentissage/ test (80 % des données/ 20 % des données)."),
    html.P("Les modèles sont entrainés sur l'ensemble d'apprentissge et testés sur l'ensemble de test"),
    html.P("La précision (Accuracy) est le quotient entre le nombre de réponse correcte et le nombre total de prédiction "),
    html.P("Les différentes méthodes qui ont été comparées sont : l'arbre décision, la forêt aléatoire, l'algorithme light gbm et l'algorithme xgboost"),
    html.P("Pour plus d'informations sur ces méthodes :"),
    html.A("Abre de décision", href = "https://www.ibm.com/fr-fr/think/topics/decision-trees"),
    html.A("Forêt aléatoire", href = "https://datascientest.com/random-forest-definition"),
    html.A("Xgboost", href = "https://xgboost.readthedocs.io/en/stable/"),
    html.A("LightGbm", href = "ttps://towardsdatascience.com/a-quick-guide-to-lightgbm-library-ef5385db8d10/"),
    
   
    
    
    
    
    dcc.Graph(id="classification-results-graph"),  # Graph for classification accuracy
    
    html.P("Ce graphique montre la précision des différentes méthodes de classifications pour chaque clustering"),
    html.P("La méthode random forest semble la plus performante"),
    html.H3("3.3 Variables les plus importantes pour la classification par tranche de prix pour random forest"),
    
    html.P("Ce graphique montre l'importance des variables pour dans la classification avec forêts aléatoires et 5 clusters"),
     # Ajouter le graphique de l'importance des variables pour les 5 clusters
    dcc.Graph(
        id="importance-variables-rf-5-clusters",
        figure=importance_fig  # La figure du graphique de l'importance des variables
    ),
    html.P("Les deux variables les plus déterminantes du prix des appartement londoniens sont la surface et la localisation (le quartier"),
    
])

    
    
    
    
    


# Callbacks pour l'application
@app.callback(
    Output('prix-maisons-graph', 'figure'),
    Input('prix-maisons-graph', 'id')
)
def update_london_graph(_):
    df_borough = medianes[medianes["Area"] == "London"]
    fig = px.line(df_borough, x="Year", y="Value", color="Measure",
                  title="Évolution des prix des maisons à Londres",
                  labels={"Value": "Prix (£)", "Year": "Année"})
    return fig

@app.callback(
    Output('graph-quartier', 'figure'),
    [Input('dropdown-quartier', 'value')]
)
def update_graph(quartier):
    df_quartier = medianes[medianes["Area"] == quartier]
    fig = px.line(df_quartier, x="Year", y="Value", color="Measure",
                  title=f"Évolution des prix à {quartier}",
                  labels={"Value": "Prix (£)", "Year": "Année"})
    return fig

@app.callback(
    Output('graph-recyclage', 'figure'),
    Input('graph-recyclage', 'id')
)
def update_recycling_graph(_):
    fig = px.bar(moyenne_recyclage, x="Recycling_Rates", y="Area",
                 title="Moyenne des taux de recyclage par quartier à Londres",
                 labels={"Recycling_Rates": "Taux moyen de recyclage (%)", "Area": "Quartier"},
                 orientation="h", color="Recycling_Rates", color_continuous_scale="viridis")
    return fig


@app.callback(
    Output('boxplot-prix', 'figure'),
    Input('boxplot-prix', 'id')
)
def update_boxplot(_):
    fig = px.box(age_batiment, x="Neighborhood", y="Price",
                 title="Distribution des prix par quartier",
                 labels={"Price": "Prix (£)", "Neighborhood": "Quartier"})
    return fig


@app.callback(
    Output('boxplot-renovation', 'figure'),
    Input('boxplot-renovation', 'id')
)
def update_renovation_boxplot(_):
    fig = px.box(age_batiment, x="Building Status", y="Price",
                 title="Distribution des prix en fonction du statut de rénovation",
                 labels={"Price": "Prix (£)", "Building Status": "Statut de rénovation"})
    return fig

@app.callback(
    Output('classification-results-graph', 'figure'),
    Input('classification-results-graph', 'id')
)
def update_classification_results(_):
    # Create a bar plot using Plotly
    fig = px.bar(df_results, 
                 x="Découpage", 
                 y="Accuracy", 
                 color="Modèle", 
                 title="Comparaison des méthodes de classification",
                 labels={"Accuracy": "Précision", "Découpage": "Clustering"},
                 barmode="group")

    # Return the figure to be displayed
    return fig
    
# Callback pour mettre à jour le tableau en fonction de la sélection
@app.callback(
    Output("cluster-table", "columns"),
    Output("cluster-table", "data"),
    Input("cluster-dropdown", "value")
)
def update_table(selected_cluster):
    df = cluster_data[selected_cluster]
    return [{"name": col, "id": col} for col in df.columns], df.to_dict('records')




if __name__ == "__main__":
    app.run_server(debug=True)


<ipython-input-6-77645c85174a>:203: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-6-77645c85174a>:204: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

